In [ ]:
# ! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/
#! cp /kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download facebook-recruiting-iii-keyword-extraction -f Train.zip

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import classification_report
import numpy as np
import time

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

#pip install scikit-multilearn --upgrade

[nltk_data] Downloading package punkt to
[nltk_data]     /N/u/tanakulk/Carbonate/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /N/u/tanakulk/Carbonate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /N/u/tanakulk/Carbonate/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /N/u/tanakulk/Carbonate/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
# ! unzip Train.zip

In [5]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# train_data = pd.read_csv("/content/drive/MyDrive/Train/Train.csv")

In [6]:
train_data = pd.read_csv("Train.csv")
len(train_data)

6034195

Dropping duplicate values present in the dataset

In [7]:
data = train_data.drop_duplicates(subset = ["Title","Tags"])
len(data)

4186184

Dropping rows with no tags

In [8]:
data['Tags'].isna().sum()

7

In [9]:
data = data.dropna(subset=['Tags'])

Using 200k records for computation purposes

In [10]:
data_split = data.loc[:100000]

In [11]:
data_split = data_split.reset_index(drop = True)

function to perform Lemmatization using WordNetLemmatizer

In [12]:
def lemmatize(name):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV

    lemmatizer = WordNetLemmatizer()

    lemmatized = []

    for i in range(len(data_split)):
      ls = ""
      for word, tag in pos_tag(data_split[name][i]):
        ls += " " + lemmatizer.lemmatize(word, tag_map[tag[0]])
      lemmatized.append(ls)
    return lemmatized

Function to complete the following pre-processnig steps:
1. Remove HTML tags using Beautiful Soup
2. Remove special characters using regular expressions
3. Convert text to its lower form
4. Tokenize the text
5. Removing stopwords except 'C' and 'R'
6. Performing Lemmatization

In [13]:
def preprocessing(columnname, name):
    
    cleantext = []
    for i in range(len(data_split)):
      cleantext.append(BeautifulSoup(data_split[columnname][i], "html").text)
    
    data_split[name] = cleantext
    
    data_split[name] = data_split[name].apply(lambda x : re.sub(r"[^a-zA-Z0-9]+", ' ', x))
    
    data_split[name] = data_split[name].apply(lambda s: s.lower() if type(s) == str else s)
    
    data_split[name] = data_split[name].apply(lambda x : word_tokenize(x))
    
    stopword = stopwords.words('english')
    data_split[name] = data_split[name].apply(lambda x : [word for word in x if not word in stopword or word=='c' or word=='r'])
    
    name_new = name + " " + "tokens"
    data_split[name_new] = lemmatize(name)


In [14]:
preprocessing('Body','clean body text')
preprocessing('Title','clean title text')
data_split

/N/u/tanakulk/Carbonate/.conda/envs/CV/lib/python3.8/site-packages/bs4/__init__.py:414: MarkupResemblesLocatorWarning: "http://127.0.0.1:8500/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


,Id,Title,Body,Tags,clean body text,clean body text tokens,clean title text,clean title text tokens
0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...,"[like, check, uploaded, file, image, file, e, ...",like check upload file image file e g png jpg...,"[check, uploaded, file, image, without, mime, ...",check upload file image without mime type
1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox,"[favorite, editor, vim, regularly, use, ctrl, ...",favorite editor vim regularly use ctrl w exec...,"[prevent, firefox, closing, press, ctrl, w]",prevent firefox close press ctrl w
2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning,"[import, matlab, file, construct, data, frame,...",import matlab file construct data frame matla...,"[r, error, invalid, type, list, variable]",r error invalid type list variable
3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding,"[probably, simple, simply, find, answer, basic...",probably simple simply find answer basicaly w...,"[replace, special, characters, url]",replace special character url
4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents,"[function, modify, mcontact, file, get, conten...",function modify mcontact file get content htt...,"[modify, whois, contact, details]",modify whois contact detail
...,...,...,...,...,...,...,...,...
99423,99997,Query data in Powerpoint,<p>I would like to build an MS PowerPoint 2010...,automation microsoft-powerpoint microsoft-powe...,"[would, like, build, ms, powerpoint, 2010, pre...",would like build m powerpoint 2010 presentati...,"[query, data, powerpoint]",query data powerpoint
99424,99998,Convolution of function,<p>I need find $x^2\cdot e^{{-x^2}/2} * e^{{-x...,functional-analysis fourier-analysis,"[need, find, x, 2, cdot, e, x, 2, 2, e, x, 2, ...",need find x 2 cdot e x 2 2 e x 2 2 used state...,"[convolution, function]",convolution function
99425,99999,How to output a value stored in a variable in ...,"<p>I am learning assembly programming using ""8...",assembly x86 dos,"[learning, assembly, programming, using, 8086,...",learn assembly program use 8086 emu software ...,"[output, value, stored, variable, assembly, 8086]",output value store variable assembly 8086
99426,100000,strange behaviour with a tableviewcontroller,"<p><img src=""http://sharingdream.com/phone.jpg...",c# iphone monotouch,"[dear, strange, problem, application, building...",dear strange problem application build use mo...,"[strange, behaviour, tableviewcontroller]",strange behaviour tableviewcontroller


Total number of tags and unique tags in 200k training data

In [15]:
def get_unique_tags(df):
    tags = df['Tags'].str.cat(sep=' ')
    tags = list(tags.split(" "))
    print("Total tags present",len(tags))
    print("Total unique tags present:",len(list(set(tags))))
    return tags

In [16]:
tags = get_unique_tags(data_split)

Total tags present 286654
Total unique tags present: 18448


In [17]:
tag_count = {}
unique_tags = list(set(tags))

for tag in unique_tags:
  #print(tag)
  tag_count[tag] = tags.count(tag)

Creating a dataframe of unique tags and the number of times each tag is appears in the answer.

In [18]:
sorted_tags = pd.DataFrame(sorted(tag_count.items(), key=lambda item: item[1],reverse=True),columns=['tags','count'])
sorted_tags

,tags,count
0,c#,7745
1,java,6753
2,php,6543
3,javascript,6102
4,android,5287
...,...,...
18443,responsetext,1
18444,scripting-languages,1
18445,cocomo,1
18446,unreachable-code,1


Dropping all tags except the top 500 tags from every record

In [19]:
top500_tags = sorted_tags['tags'][:500]
type(top500_tags)
type(top500_tags)
top500_tags_list = list(top500_tags)

#top500_tags_list

In [20]:
def find_most_frequent_tags(x):
  #print(x)
  list1 = x.split(" ")
  result = ""
  for tag in list1:
    #print(tag)
    if tag in top500_tags_list:
      #print("in")
      result+= " " + tag
    return result


In [21]:
data_tags = data_split['Tags'].apply(lambda x: find_most_frequent_tags(x))
print("after taking top 500 frequency tags",data_tags.shape)
data_tags = data_tags.apply(lambda x: x if len(x)>0 else None)
data_new = pd.concat([data_split[['clean title text tokens','clean body text tokens']],data_tags],axis=1)
print(data_new.shape)
print(data_new)
data_new.reset_index(drop=True)
data_new.dropna(subset= ['Tags'],inplace = True)
print("After dropping tags with 0 length",data_new.shape)
data_new.reset_index(drop= True)
print("data new is", data_new)
type(data_new.head())

after taking top 500 frequency tags (99428,)
(99428, 3)
                          clean title text tokens  \
0       check upload file image without mime type   
1              prevent firefox close press ctrl w   
2              r error invalid type list variable   
3                   replace special character url   
4                     modify whois contact detail   
...                                           ...   
99423                       query data powerpoint   
99424                        convolution function   
99425   output value store variable assembly 8086   
99426       strange behaviour tableviewcontroller   
99427       need advice start cod next iphone app   

                                  clean body text tokens                  Tags  
0       like check upload file image file e g png jpg...                   php  
1       favorite editor vim regularly use ctrl w exec...               firefox  
2       import matlab file construct data frame matla...        

pandas.core.frame.DataFrame

Vectorizing labels (tags) with 0 or 1 (binary vectorizer)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vec = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
y = vec.fit_transform(data_new['Tags'])

Feature extracting using tfidf for both body and titles. We get sparse matrices as output after fit and transform.

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_body = TfidfVectorizer(min_df=0.00009, max_features=20000, smooth_idf=True, norm="l2",tokenizer = lambda x: x.split(), sublinear_tf=False,ngram_range=(1,4))
tfidf_title = TfidfVectorizer(min_df=0.00009, max_features=20000, smooth_idf=True, norm="l2",tokenizer = lambda x: x.split(), sublinear_tf=False,ngram_range=(1,4))
x_train_tfidf_body = tfidf_body.fit_transform(data_new['clean body text tokens'])
x_train_tfidf_title = tfidf_title.fit_transform(data_new['clean title text tokens'])

Stacking body and title together

In [24]:
from scipy.sparse import hstack
x_data = hstack([x_train_tfidf_body,x_train_tfidf_title])

Splitting data into train and test

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train, y_test = train_test_split(x_data, y, test_size=0.2,random_state = 0)

In [26]:
print("shape of train data X:",x_train.shape, "Y :",y_train.shape)
print("shape of test data X:",x_test.shape,"Y:",y_test.shape)

shape of train data X: (70353, 29428) Y : (70353, 493)
shape of test data X: (17589, 29428) Y: (17589, 493)


Training and testing a Binary Relevance classifier using Gaussian Naive Bayes

In [27]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

startbr=time.time()

brn = BinaryRelevance(GaussianNB())

brn.fit(x_train, y_train)

print("Time taken to train :", round(time.time()-startbr,0))

Time taken to train : 32365.0


In [28]:
brn_pred = brn.predict(x_test)

print("Accuracy: ",accuracy_score(y_test, brn_pred))
print("Hamming loss: ",hamming_loss(y_test,brn_pred))
# print("Jaccard Score: ", jaccard_score(y_test, brn_pred), average='samples')

Accuracy:  0.1410540678833362
Hamming loss:  0.003445934826729365


In [29]:
from sklearn.metrics import classification_report

print (classification_report(y_test, brn_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.07      0.07      0.07       230
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00         5
           6       0.44      0.11      0.18        35
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00        48
           9       0.00      0.00      0.00        14
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00        29
          15       0.00      0.00      0.00        25
          16       0.25      0.02      0.04        48
          17       0.00    

/N/u/tanakulk/Carbonate/.conda/envs/CV/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/N/u/tanakulk/Carbonate/.conda/envs/CV/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/N/u/tanakulk/Carbonate/.conda/envs/CV/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(

Predict a sample

In [63]:
# test1 = ['problem eclipse regard xml file eclipse complain android scrollbars android fadingedge allow string check android developer site fact accept string xml file relate question pose problem miss android xmlns see code line begin xmlns correct believe complete file content xml version 1 0 encode utf 8 com example todolist todolistitemview xmlns android http schema android com apk res android android layout width fill parent android layout height fill parent android pad 10dp android scrollbars verticle android textcolor color notepad text android fadingedge verticle']
# tfidf_test = TfidfVectorizer(min_df=0.00009, max_features=20000, smooth_idf=True, norm="l2",tokenizer = lambda x: x.split(), sublinear_tf=False,ngram_range=(1,4))
# test2 = ['eclipse complain android scrollbars android fadingedge allow string include code']
test_sent = ["import pandas as pd import as numpy np FutureWarning: Pass n_neighbors=8 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error will result in an error"]
test_sent1=["python numpy error at runtime"]

qs = tfidf_body.transform(test_sent)

qs2 = tfidf_title.transform(test_sent1)


q = hstack([qs,qs2])
q.shape

(1, 29428)

In [64]:
op = brn.predict(q)
print(op)

  (0, 342)	1


In [53]:
print(vec.get_feature_names()[342])

python
